In [7]:

import pandas

import matplotlib.pyplot as plt

 

from ipywidgets import widgets

from IPython.display import display

from io import BytesIO

 

uploader = widgets.FileUpload( accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description='upload csv',
    button_style='info'
    )

 

dropdown = widgets.Dropdown(

    options=['none'*7],

    # description='browse csv',


    )

 

button = widgets.Button(

    description='calculate stats',


    button_style='', # 'success', 'info', 'warning', 'danger' or ''

    tooltip='statistics',

    icon='calculator' # (FontAwesome names without the `fa-` prefix)

    )

button2 = widgets.Button(

    description='plot graphs',


    button_style='', # 'success', 'info', 'warning', 'danger' or ''

    icon='line-chart' # (FontAwesome names without the `fa-` prefix)

    )

 

output = widgets.Output()

output2 = widgets.Output()

hbox = widgets.HBox(children=[uploader,dropdown, button, button2])

outbox = widgets.HBox(children=[output, output2])

vbox = widgets.VBox(children=[hbox,outbox])

display(vbox)

 

df_dict={}

df_dict2={}

 

def handle_upload(change):

    if not change.new:

        return

    up = change.owner

    for filename,data in up.value.items():

        if filename not in dropdown.options:

            dropdown.options += (filename,)

            df = pandas.read_csv(BytesIO(data['content']))

           

            try:

                for i in range(len(df.index)):

                    df.iloc[i,:] = df.iloc[i,:].str.replace(",", ".").astype(float)

            except Exception as e:
                print(e.__traceback__)

                # pass

           

            df_dict[filename]=df

            dropdown.options = list(df_dict.keys())  

           

            dropdown.value = filename

           

       

        # print(f'writing [{filename}] to ./')

        # with open(filename, 'wb') as f:

            # f.write(data['content'])

    up.value.clear()

    up._counter = 0

   

    dropdown.disabled = False

   

   

def handle_selection(change):

    if(dropdown.value):
        if(dropdown.value != 'none'):
            button.disabled = False
            button.button_style = 'success'
            df=df_dict[dropdown.value]
            output.clear_output()

            with output:
                display(df)
        else:

            button.disabled = True
            button.button_style = ''
    # with output:

    #     # print(change.keys())

    #     print(change['new'])

   

   

def calculateStats(b):

    button.button_style = 'info'

    button.disabled = True

   

    button2.button_style = 'success'

    button2.disabled = False

   

    df=df_dict[dropdown.value]

    df2=pandas.DataFrame(columns=df.columns)

    df2.loc['median'] = df.median()

    df2.loc['mean'] = df.mean()

    df2.loc['peak'] = df.max()

    df2.loc['std'] = df.std()

   

    output2.clear_output()

    with output2:

        display(df2)

   

    # with output:

    #     print(b)

   

def plotGraphs(b):

    button2.button_style = 'info'

    button2.disabled = True

   

    df=df_dict[dropdown.value]

    for k, c in enumerate(df.columns):

        plt.figure(k)

        df[c].plot(title=c)

 

    # with output:

    #     print(b)

uploader.observe(handle_upload, names='value')

dropdown.observe(handle_selection, names='value')

button.on_click(calculateStats)

button2.on_click(plotGraphs)

FileUpload(value={}, accept='.csv', button_style='info', description='upload csv')